# Install all the Required Libraries

In [ ]:
import tensorflow
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11725400464976449446
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14415560704
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5006745974174944383
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

# Install Detectron2(object Detection Model)

In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-c6osafy0
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-c6osafy0
     |████████████████████████████████| 50 kB 7.2 MB/s 
     |████████████████████████████████| 79 kB 8.5 MB/s 
     |████████████████████████████████| 151 kB 61.5 MB/s 
     |████████████████████████████████| 1.4 MB 50.6 MB/s 
     |████████████████████████████████| 548 kB 67.2 MB/s 
     |████████████████████████████████| 248 kB 64.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 843 kB 54.3 MB/s 
     |████████████████████████████████| 96 kB 7.4 MB/s 
     |████████████████████████████████| 117 kB 57.9 MB/s 
     |████████

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import pandas as pd
import tqdm
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog
import time

# Git Clone Tracking Function called SORT

In [ ]:
!git clone https://github.com/abewley/sort.git

Cloning into 'sort'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 205 (delta 0), reused 1 (delta 0), pack-reused 202
Receiving objects: 100% (205/205), 1.20 MiB | 28.04 MiB/s, done.
Resolving deltas: 100% (72/72), done.


# There is aPackaged version Available for people Who wants to pip install
For those use the command Below

pip install sort-track 


# Install filterpy to use Kalman filters for SORT Algo

In [ ]:
!pip install filterpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 25.8 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=dfe04ca2d7b9bd5495e3387403740759055fc9df29991a61c79b1a869c5b9a72
  Stored in directory: /root/.cache/pip/wheels/ce/e0/ee/a2b3c5caab3418c1ccd8c4de573d4cbe13315d7e8b0a55fbc2
Successfully built filterpy


In [ ]:
from sort.sort import Sort  #Run  this cell 2 times


# Count Function and Function to find Center of Bounding-Box

In [ ]:
def find_center(x, y, w, h):
    x1=int(w/2)
    y1=int(h/2)
    cx = x+x1
    cy=y+y1
    return cx, cy
def count_vehicle(box_id, img):
    global Lcount,Rcount
                                                               
    x, y, w, h, id = box_id
    index=0
    # Find the center of the rectangle for detection
    center = find_center(x, y, w, h)
    ix, iy = center
    
    # Find the current position of the vehicle
    if (iy > up_line_position) and (iy < middle_line_position):
        if id not in temp_up_list:
            temp_up_list.append(id)
                                                                             
    elif iy < down_line_position and iy > middle_line_position:
        if id not in temp_down_list:
            temp_down_list.append(id)
                                                         
    elif iy < up_line_position:
        if id in temp_down_list:
            temp_down_list.remove(id)
            up_list[index] = up_list[index]+1
                                                             
            Lcount=Lcount+1     
    elif iy > down_line_position:
        if id in temp_up_list:
            temp_up_list.remove(id)
            down_list[index] = down_list[index]
                                                            
            Rcount=Rcount+1     
     

#Using Detectron2 Model with
# OUTPUT:


*  Csv file containing Type of vehicle,B-BOX ID,Detection Accuracy,Video Frame No,Video Time(Seconds)
*  shows Realtime Object Tracking and Counting of vehicles in left lane of Road and right lane in Video(may lag due to performnce of system)





In [ ]:


# Extract video properties
samp='/content/sample_traffic.mp4'
dhi='/content/drive/MyDrive/Heavy traffic seen at ISBT Kashmiri Gate, New Delhi  Date 15-05-2022.mp4'
video = cv2.VideoCapture(samp)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
print(frames)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
sucess, img = video.read()
font_color = (0, 0, 255)
font_size = 0.4
font_thickness = 1
temp_up_list = []
temp_down_list = []
up_list = [0, 0, 0, 0]
down_list = [0, 0, 0, 0]
Lcount=0
Rcount=0
#print(num_frames)

# Initializframes_per_seconde video writer

# Initialize predictor

middle_line_position = 500   
up_line_position = middle_line_position - 40
down_line_position = middle_line_position + 40

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
#cfg.MODEL.DEVICE = 'cpu'
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
mot_tracker = Sort(max_age=2, min_hits=1, iou_threshold=0.15)
count=1
# Initialize visualizer
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
time=0
#Empty dataset to append to
new_dta=pd.DataFrame()
while sucess:
          #print('%d:%02d:%02d' % (hour, min, sec))
         # img = cv2.resize(img,(0,0),None,0.5,0.5)
          #ih, iw, channels = img.shape

          outputs = predictor(img)
          pt=[]
          st=[]
          res=[]
          for point in outputs["instances"].pred_boxes:
            sr=[]
            sr.append(point.tolist())
            pt.append(sr)
          for sc in outputs["instances"].scores.tolist():
              st.append(sc)
          for i in range(len(outputs["instances"].scores)):
                for it in pt[i]:
                    it.append(st[i]*100)
                    #print(it)
                    res.append(it)
        
          res = np.array(res)
          if len(res)==0:
              res=np.empty((0, 5))
          track_bbs_ids = mot_tracker.update(res)
          for box_id in track_bbs_ids:
            count_vehicle(box_id, img)
             
          #Make sure the frame is colored
          #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

          #Draw a visualization of the predictions using the video visualizer
          visualization = v.draw_instance_predictions(outputs["instances"].to("cpu"))


          # creating CSV 
          min, sec = divmod(time, 60)
          hour, min = divmod(min, 60)
          #print("frame-",count)
          vftime='%d:%02d:%02d' % (hour, min, sec)
          rem=[]
          for data in outputs["instances"].pred_classes:
            num = data.item()
            cl=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes[num]
            rem.append(cl)
          
          dta=pd.DataFrame(data=rem,columns={"Vehicle Type"})
          dta['Box ID']=pd.Series(track_bbs_ids[:,4]) #for including absent ids as nan values Series is used
          dta['detection_score']=st
          dta['frame no']=count
          dta['video_frame_time']=vftime
          #print(dta)
          
          new_dta=pd.concat([new_dta,dta])
          count=count+1
          if count%25==0: time=time+1
          
          sucess,img=video.read()
          


new_dta.to_csv("finalvehicledata.csv",index=False)

# Release resources
video.release()
cv2.destroyAllWindows()

511.0


#Using Detectron2 Model with
# OUTPUT:


*  Video File is Created which shows Realtime Object Tracking and Counting of vehicles in left lane of Road and right lane. 

we can Download the video(No lag)






In [ ]:
samp='/content/sample_traffic.mp4'
dhi='/content/drive/MyDrive/Heavy traffic seen at ISBT Kashmiri Gate, New Delhi  Date 15-05-2022.mp4'
video = cv2.VideoCapture(samp)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
sucess, img = video.read()
font_color = (0, 0, 255)
font_size = 0.4
font_thickness = 1
temp_up_list = []
temp_down_list = []
up_list = [0, 0, 0, 0]
down_list = [0, 0, 0, 0]
Lcount=0
Rcount=0
print(num_frames)
# Initializframes_per_seconde video writer

video_writer = cv2.VideoWriter('0.25---5==2FINAL_count.mp4', fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=25.0, frameSize=(width, height), isColor=True)
# Initialize predictor

middle_line_position = 500   
up_line_position = middle_line_position - 40
down_line_position = middle_line_position + 40

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
#cfg.MODEL.DEVICE = 'cpu'
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
mot_tracker = Sort(max_age=5, min_hits=2, iou_threshold=0.25)

# Initialize visualizer
v = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), ColorMode.IMAGE)

def generate_frame(video):
        
        sucess=True
        while sucess:
          sucess,img=video.read()
          if not sucess:
             break

         # img = cv2.resize(img,(0,0),None,0.5,0.5)
          ih, iw, channels = img.shape
          outputs = predictor(img)
          pt=[]
          st=[]
          res=[]
          for point in outputs["instances"].pred_boxes:
            sr=[]
            sr.append(point.tolist())
            pt.append(sr)
          for sc in outputs["instances"].scores.tolist():
              st.append(sc)
          for i in range(len(outputs["instances"].scores)):
                for it in pt[i]:
                    it.append(st[i]*100)
                    #print(it)
                    res.append(it)
        
          res = np.array(res)
          if len(res)==0:
              res=np.empty((0, 5))
          track_bbs_ids = mot_tracker.update(res)
          for box_id in track_bbs_ids:
            count_vehicle(box_id, img)
            #print(type(int(box_id[4])))  
          #Make sure the frame is colored
          img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

          #Draw a visualization of the predictions using the video visualizer
          visualization = v.draw_instance_predictions(img, outputs["instances"].to("cpu"))
          #print(type(visualization))
          #Convert Matplotlib RGB format to OpenCV BGR format
          img = cv2.cvtColor(visualization.get_image(), cv2.COLOR_BGR2RGB)
          """ct=0
          for box in track_bbs_ids:
            b4=int(box_id[4])
            cv2.putText(img,"VEH"+str(ct),(int(box[0]), int(box[1])), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
            ct=ct+1"""
          cv2.putText(img, "LeftCount="+str(Lcount), (110, 20), cv2.FONT_HERSHEY_TRIPLEX, font_size, font_color, font_thickness)
          cv2.putText(img, "RightCount="+str(Rcount), (110, 40), cv2.FONT_HERSHEY_TRIPLEX, font_size, font_color, font_thickness)
          cv2.line(img, (0, middle_line_position), (iw, middle_line_position), (255, 0, 255), 1)
          cv2.line(img, (0, up_line_position), (iw, up_line_position), (0, 0, 255), 1)
          cv2.line(img, (0, down_line_position), (iw, down_line_position), (0, 0, 255), 1)
          #cv2_imshow(img)
          yield img

for  img in generate_frame(video):
  video_writer.write(img)

# Release resources
video.release()
video_writer.release()
cv2.destroyAllWindows()